<a href="https://colab.research.google.com/github/kentokura/gob/blob/main/gobgob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [197]:
#  id
# ( R or B )+ ( a ~ i , z 及びその大文字からなる12文字)
#  id 例 : Rzzzzzzzzzzzz

# choise_piece, board
# [RL, RM, RS, BL, BM, BS, hand] と並んでいる。
# RL ~ BS は[0,0,0,0,0,0,0,0,0] で、[a~i]のマスの場所を表す
# handは[2,2,2,2,2,2]で、RL...BSの順に持ち駒の数を表す

In [202]:
# id , choise_piece, boardの相互変換

# アルファベットを数字に変換 a -> 0
alpha2num = lambda c: ord(c) - ord('a')
# 数字をアルファベットに変換 0 -> a
num2alpha = lambda c: chr(c+64)
# 数字と場所の対応表
num2place_dict = {0:'a', 1:'b', 2:'c', 3:'d', 4:'e', 5:'f', 6:'g', 7:'h', 8:'i'}

# idからchoise_pieceを作成
def id2choise_piece(id):
  choise_piece = [
          [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0]
          ]
  if id.islower() == True:
    return choise_piece
  for i, piece in enumerate(id[1:]):
    pieces_type = i // 2
    # 大文字をchoise_pieseに反映
    if piece.isupper() == True:
      if piece == "Z":
        choise_piece[6][pieces_type] = 1
      else:
        choise_piece[pieces_type][alpha2num(piece.lower())] = 1
      return choise_piece

# idからboardを作成
def id2board(id):
  board = [
          [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0]
          ]
  for i, piece in enumerate(id[1:]):
    pieces_type = i // 2
    if piece.isupper() == True:
      continue;
    if piece == "z":
      board[6][pieces_type] += 1
    else: # piece == a ~ i
      board[pieces_type][alpha2num(piece.lower())] = 1
  return board

# color, choise_piece, boardからidへ変換
def create_id(color, choise_piece, board):
  color = color
  choise_piece = choise_piece
  board = board
  pieces = ""
  str = ["zz","zz","zz","zz","zz","zz"]
  # boardをpiecesに反映
  for i, strings in enumerate(board[:6]):
    str[i] = ""
    # piecesを前から更新していく
    # num2place_dictのkeyと[0,0,...,0]の1の位置の数が同じkeyを取得
    for j, p in enumerate(strings):
      if p == 1:
        str[i] += num2place_dict[j]
    str[i] = str[i].ljust(2, "z")
  pieces = ''.join(str)
  # choise_pieceをpiecesに反映
  if choise_piece == [
              [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0]
          ]:
    pass;
  else: 
    # [:6]まで対応
    for i, strings in enumerate(choise_piece[:6]):
      for j, p in enumerate(strings):
        if p == 1:
          char = num2place_dict[j].upper()
          pieces = list(pieces)
          pieces[i*2+1] = char
          pieces = "".join(pieces)
          break;
    # handの対応
    for i, q in enumerate(choise_piece[6]):
      if q == 1:
        char = "Z"
        pieces = list(pieces)
        pieces[i*2+1] = char
        pieces = "".join(pieces)
  id = color + pieces
  return id

In [236]:
class State:
  # 初期化
  def __init__(self, id=None):
    self.id = id if id != None else "rzzzzzzzzzzzz"
    self.mode = "put" if self.id == self.id.lower() else "choise"
    self.pieces = self.id[1:]
    self.color = self.id[0]
    # [RL, RM, RS, BL, BM, BS, hand]
    self.choise_piece = [
            [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
            [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
            [0,0,0,0,0,0]
            ]
    self.board = [
            [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
            [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
            [0,0,0,0,0,0]
            ];
    self.choise_piece = id2choise_piece(self.id)
    self.board = id2board(self.id)

  # 確認
  def __str__(self):
    str = "mode\t= {}\n".format(self.mode)
    str += "pieces\t= {}\n".format(self.pieces)
    str += "color\t= {}\n".format(self.color)
    str += "choise_piece\t= {}\n".format(self.choise_piece)
    str += "board\t= {}\n".format(self.board)
    return str

In [237]:
# 合法手
# actions = [action1, action2,...]
# action = [color, choise_piece, board]
def legal_actions(state):
  actions = ""
  # action = choiseするなら12種類, putするなら9種類
  if state.mode == "choise":
    # choise_pieceをputする
    print("hoge")
  elif state.mode == "put":
    # boardからchoiseする
    print("huga")
  return actions


In [239]:
state = State("rzzzzzzZzzzzz")
print(state.mode)
print(legal_actions(state))

choise
hoge

